In [3]:
import numpy as np
def costheta(a,b):
    return (np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)))

#This will be a function in Crystal class
def pair_gen(self,chem,or_pure,or_mixed):
    """ Takes in preferred orientations of dumbbells in pure and mixed state and pairs them with the site index
        Param: chem - index of sublattice to consider dumbbell diffusion in.
        Param: or_pure - list of numpy arrays containing pure dumbbell orientations for the sites.
        Param: or_mixed - list of numpy arrays containing mixed dumbbell orientations for the sites.
        
        Output: pairs : list of lists - each list is of the form [index,symm_or_pure,symm_or_mixed]
                        where index - basis atom index for the given sublattice
                        symm_or_* - a list of symmetry equivalent orientations prefferred for that site."""
    #Checking if correct sublattice index has been provided
    if chem >= len(self.basis):
        raise IndexError("sublattice index 'chem' is out of range")
    n = len(self.basis.chem)
    #Checking if inputs have correct format
    if not isinstance(or_pure,list) or not isinstance(or_mixed,list) :
        raise TypeError('Orientations need to be given as a list of numpy arrays')
    if not (len(or_pure)==n) or not (len(or_mixed)==n):
        raise TypeError('Number of supplied orientations needs to be the same as number of basis atoms')
    for i in range(n):
        if not isinstance(or_pure[i],(list,np.ndarray)) or not not isinstance(or_mixed[i],(list,np.ndarray)):
            raise TypeError("Orientation vectors must be numpy arrays")
    or_pure_all=[] #list of lists - symetrically equivalent directions corresponding to each preferred orientation.
    or_mixed_all=[]
    for i in range(n):
        or_pure_temp = []
        or_mixed_temp = []
        
        for g in self.G:
            #Ends of vectors will be explicitly identified with + or - 1, so don't need to store negatives.
            purevec=g.cartrot(or_pure[i])
            if not any(np.allclose(costheta(x,purevec),-1.0) for x in or_pure_temp):
                or_pure_temp.append(purevec)
                
            mixedvec=g.cartrot(or_mixed[i])
            if not any(np.allclose(costheta(x,mixedvec),-1.0) for x in or_mixed_temp):
                or_mixed_temp.append(mixedvec)
                
        or_pure_all.append(or_pure_temp)
        or_mixed_all.append(or_mixed_temp)
    
    final = []
    for i in range(n):
        t = [self.basis.chem[i],or_pure_all[i],or_mixed_all[i]]
        # t is a list in "final" - contains basis site, all directions of pure dumbbell, all directions of mixed dumbbell for that site.
        final.append(t) #final - big list of lists of "type" t.
    return (final)
#Users can modify the list "final" before creating the jumpnetwork.    

In [1]:
#This will be a function in Crystal class
def jumpnetwork_db(self, pairs, cutoff, closestdistance=0):
    """
         Follows "jumpnetwork" function in crystal.py
        :param pairs: list that contains basis atoms and orientation vectors grouped together.
                      The list 'final' that is returned from pair_gen.
        :param cutoff: distance cutoff
        :param c - cartesian vector that is a member of the family of directions which can be orientations
         of the dumbbell
        :param closestdistance: closest distance allowed in transition (can be a list) do we need this for dumbbells?
        """
    def compare(tup1,tup2): #to compare tuples of format (index,nparray3,tag,+-1)
                            #tag-an integer to keep track of which set the orientation (nparray3) comes from
        return(tup1[0]==tup2[0] and np.allclose(tup1[1],tup2[1]) and tup1[2]==tup2[2] and tup1[3]==tup2[3])

    def inlist(tup, dx, lis):
            return any(compare(tup[0],ij[0]) and compare(tup[1],ij[1]) and self.__isclose__(dx, v) for translist in lis for ij, v in translist)

    r2 = cutoff**2
    nmax = [int(np.round(np.sqrt(self.metric[i, i]))) + 1 for i in range(3)]
    supervect = [np.array([n0, n1, n2])
                     for n0 in range(-nmax[0], nmax[0] + 1)
                     for n1 in range(-nmax[1], nmax[1] + 1)
                     for n2 in range(-nmax[2], nmax[2] + 1)]
    lis = []
    center = np.zeros(3, dtype=int)
    for i,l0 in enumerate(pairs):
        for j,l1 in enumerate(pairs):
            u0=l0[0] #same as crys.basis[chem][i]
            u1=l1[0] #same as crys.basis[chem][j]
            or0_p = l0[1]
            or1_p = l1[1]
            or0_m = l0[2]
            or1_m = l1[2]
            du = u1-u0
            for n in supervect:
                    dx = self.unit2cart(n, du)
                    if (np.linalg.norm(dx)<=1e-8): #If no movement, choose from same set.
                        #First do pure orientations (tag=0)
                        for o0 in or0_p:
                            for o1 in or1_p:
                                if not (np.allclose(o0,o1)): #do not make a diagonal state #use default tolerance for now.
                                    for x in [-1,1]:
                                        for y in [-1,1]:
                                            if not inlist(((i,o0,0,x),(j,o1,0,y)),dx,lis):#tag = 0
                                                trans = []
                                                for g in self.G:
#                                                     R0, ind0 = self.g_pos(g, center, (chem, i)) #go to line 1169 in crystal.py
#                                                     #to see what g_pos returns
#                                                     R1=R0.copy() #n is [0,0,0] as well - same as center
#                                                     ind1=ind0 #using copy to avoid problems with mutable types
                                                    o0_new = np.dot(g.cartrot,o0)
                                                    o1_new = np.dot(g.cartrot,o1)
                                                    tup0 = (i,o0_new,0,x)
                                                    tup1 = (i,o1_new,0,y) #|dx|=0 implies i=j. We are rotating about in the same site.
                                                    #Now check 180 degree cases
                                                    for o0_test in or0_p:
                                                        if np.allclose(costheta(o0_test,o0_new),-1.):
                                                            tup0 = (i,o0_test,0,-x)
                                                    for o1_test in or1_p:
                                                        if np.allclose(costheta(o1_test,o1_new),-1.):
                                                            tup1 = (i,o1_test,0,-y)
                                                    #Check if we have no actual rotation
                                                    if not compare(tup0,tup1):
                                                        tup = (tup0,tup1)
                                                        if not any(compare(tup[0],ij[0]) and compare(tup[1],ij[1]) and self.__isclose__(dx, v) for ij, v in trans):
                                                            trans.append((tup,dx))
                                                            trans.append(((tup[1], tup[0]), -dx))
                                                lis.append(trans)
                        #Now do mixed dumbbell rotations - tag=1
                        for o0 in or0_m:
                            for o1 in or1_m:
                                if not (o0 == o1): #do not make a diagonal state
                                    for x in [-1,1]:
                                        for y in [-1,1]:
                                            if not inlist(((i,o0,1,x),(j,o1,1,y)),dx,lis):#dx = 0
                                                trans = []
                                                for g in self.G:
#                                                     R0, ind0 = self.g_pos(g, center, (chem, i)) #line 1169 in crystal.py
#                                                     #to see what g_pos returns
#                                                     R1=R0.copy() #n is [0,0,0] as well - same as center
#                                                     ind1=ind0
                                                    o0_new = np.dot(g.cartrot,o0)
                                                    o1_new = np.dot(g.cartrot,o1)
                                                    tup0 = (i,o0_new,1,x)
                                                    tup1 = (i,o1_new,1,y)#|dx|=0 implies i=j. We are rotating about in the same site.
                                                    #Now check 180 degree cases
                                                    for o0_test in or0_p:
                                                        if np.allclose(costheta(o0_test,o0_new),-1.):
                                                            tup0 = (i,o0_test,1,-x)
                                                    for o1_test in or1_p:
                                                        if np.allclose(costheta(o1_test,o1_new),-1.):
                                                            tup1 = (i,o1_test,1,-y)
                                                    #Check if we have no actual rotation
                                                    if not (tup0==tup1):
                                                        tup = (tup0,tup1)
                                                        if not any(compare(tup[0],ij[0]) and compare(tup[1],ij[1]) and self.__isclose__(dx, v) for ij, v in trans):
                                                            trans.append((tup,dx))
                                                            trans.append(((tup[1], tup[0]), -dx))
                                                lis.append(trans)
                    
                    else: #If there is movement, choose from both sets.
                    #tags inserted to see which set an orientation comes from.
                        for count0, o0 in enumerate(or0_p+or0_m):
                            for count1,o1 in enumerate(or1_p+or1_m):    
                                tag0 = int(count0/len(or0_p)) #0 if pure, 1 if mixed.
                                tag1 = int(count1/len(or1_p)) #0 if pure, 1 if mixed.
                                for x in [-1,1]:
                                    for y in [-1,1]:
                                        if not inlist(((i,o0,tag0,x),(j,o1,tag1,y)),dx,lis):
                                            trans = []
                                            for g in self.G:
                                                R0, ind0 = self.g_pos(g, center, (chem, i)) #line 1169 in crystal.py
                                                R1, ind1 = self.g_pos(g, n, (chem, j))
                                                o0_new = np.dot(g.cartrot,o0)
                                                o1_new = np.dot(g.cartrot,o1)
                                                tup0 = (ind0[1],o0_new,tag0,x)
                                                tup1 = (ind1[1],o1_new,tag1,y)
                                                #Now check 180 degree cases
                                                for o0_test in (or0_p+or0_m):
                                                    if np.allclose(costheta(o0_test,o0_new),-1.):
                                                        tup0 = (ind0[1],o0_test,tag0,-x)
                                                for o1_test in (or0_p+or0_m):
                                                    if np.allclose(costheta(o1_test,o1_new),-1.):
                                                        tup1 = (ind1[1],o1_test,tag1,-y)
                                                tup = (tup0,tup1)
                                                dx = self.pos2cart(R1, ind1) - self.pos2cart(R0, ind0)
                                                if not any(compare(tup[0],ij[0]) and compare(tup[1],ij[1]) and self.__isclose__(dx, v) for ij, v in trans):
                                                    trans.append((tup,dx))
                                                    trans.append(((tup[1], tup[0]), -dx))
                                            lis.append(trans)

    #Collision detection should be the same as that for vacancies as jumps are essentially site to site.
    #I may need to modify this if physical situations dictate otherwise.
    if type(closestdistance) is list:
            closest2list = [x ** 2 if c != chem else -1. for c, x in enumerate(closestdistance)]
    else:
        closest2list = [closestdistance ** 2 if c != chem else -1. for c in range(self.Nchem)]
    for c, mindist2 in enumerate(closest2list):
        if mindist2 < 0:
            continue
        for u0 in self.basis[c]:
            for n in supervect:
                for ntrans in range(len(lis)-1,-1,-1):
                    trans = lis[ntrans]
                    t = trans[0]  # representative transition
                    dx = t[1]
                    if dx.dot(dx) > 1e-8:
                        #Avoid check if there is only rotations in the same site.
                        xRa = self.unit2cart(n, u0 - self.basis[chem][t[0][0][0]])
                        # t has format (((i,or1,tag,+-1),(j,or2,tag,+-1)),dx)
                        xRa2 = np.dot(xRa, xRa)
                        xRa_dx = np.dot(xRa, dx)
                        dx2 = np.dot(dx, dx)
                        if 0 <= xRa_dx <= dx2:
                            d2 = (xRa2 * dx2 - xRa_dx * xRa_dx) / dx2
                            #For my understanding:
                            #Draw a vector "xRa" and a vector "dx".
                            #Draw a line perpendicular to "dx" and intersecting "xRa"
                            #at the front tip of "xRa". d2 is then the squared length of this line.
                            #d2 is the distance of approach of dx to xRa.
                            if np.isclose(d2, mindist2) or d2 < mindist2:
                                # lis.remove(trans)
                                lis.pop(ntrans)

    lis.sort(key=lambda entry: min(i + j + 1e-3 * np.dot(dx, dx) for (i, j), dx in entry))
    #Reasoning behind this operation?
    return lis